In [ ]:
import math

import cv2
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from skimage import filters, feature
from skimage.color import rgb2gray
from skimage.transform import resize

In [ ]:
def nonmaxima_suppression(grad_img, angles):
    rows, cols = grad_img.shape
    
    new_img = grad_img.copy()
    
    d = np.array([0.0, math.pi/4., math.pi/2., 3. * math.pi/4.])
    neighbor_idxs = np.array([
        [0, 1],
        [1, 1],
        [1, 0],
        [1, -1]
    ])
    
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            if np.isnan(angles[i, j]):
                continue
                
            theta = angles[i, j]
            if theta < 0:
                theta += math.pi
            
            # Discrete method
            angle_diff = np.abs(d - theta)
            idx = np.argmin(angle_diff)
            neighbor_loc = neighbor_idxs[idx]
            
            n1_grad = grad_img[i + neighbor_loc[0], j + neighbor_loc[1]]
            n2_grad = grad_img[i - neighbor_loc[0], j - neighbor_loc[1]]
            grad = grad_img[i, j]
            
            max_grad = np.argmax([grad, n1_grad, n2_grad])
            
            if max_grad != 0:
                new_img[i, j] = 0
                
            #Bilinear Interpolation
#             grad = grad_img[i, j]
#             y_diff = np.sin(theta)
#             x_diff = np.cos(theta)
#             disp = np.array([y_diff, x_diff])
#             raw_coord1 = np.array([i, j]) + disp
#             raw_coord2 = np.array([i, j]) - disp
                        
#             # Interpolate value for raw_coord1
#             xmin = math.floor(raw_coord1[1])
#             xmax = math.ceil(raw_coord1[1])
#             ymin = math.floor(raw_coord1[0])
#             ymax = math.floor(raw_coord1[0])
#             wy = raw_coord1[1] - ymin
#             wx = raw_coord1[1] - xmin
#             top_left = grad_img[ymin, xmin]
#             top_right = grad_img[ymin, xmax]
#             bottom_left = grad_img[ymax, xmin]
#             bottom_right = grad_img[ymax, xmax]
#             n1_grad = (1 - wx) * (1 - wy) * top_left + \
#                       wx * (1 - wy) * top_right + \
#                       (1 - wx) * wy * bottom_left + \
#                       wx * wy * bottom_right
            
#             # Interpolate value for raw_coord2
#             xmin = math.floor(raw_coord2[1])
#             xmax = math.ceil(raw_coord2[1])
#             ymin = math.floor(raw_coord2[0])
#             ymax = math.floor(raw_coord2[0])
#             wy = raw_coord2[1] - ymin
#             wx = raw_coord2[1] - xmin
#             top_left = grad_img[ymin, xmin]
#             top_right = grad_img[ymin, xmax]
#             bottom_left = grad_img[ymax, xmin]
#             bottom_right = grad_img[ymax, xmax]
#             n2_grad = (1 - wx) * (1 - wy) * top_left + \
#                       wx * (1 - wy) * top_right + \
#                       (1 - wx) * wy * bottom_left + \
#                       wx * wy * bottom_right
            
#             max_grad = np.argmax([grad, n1_grad, n2_grad])
            
#             if max_grad != 0:
#                 new_img[i, j] = 0
                
    return new_img

def connectivity(low_img, high_img):
    nrows, ncols = low_img.shape
    
    connected_img = high_img.copy()
    
    for i in range(nrows):
        for j in range(ncols):
            if low_img[i, j] == 0:
                continue
                
            for ih in range(-1, 2):
                if i + ih < 0 or i + ih >= nrows:
                    continue
                for jh in range(-1, 2):
                    if j + jh < 0 or j + jh >= ncols:
                        continue
                    if connected_img[i + ih, j + jh] > 0:
                        connected_img[i, j] = low_img[i, j]
                        
    return connected_img

In [ ]:
# Testing connectivity
img_arr_low = np.zeros((10, 10))
img_arr_high = np.zeros((10, 10))
img_arr_low[5, ::2] = 1
img_arr_low[3, :] = 1
img_arr_high[5, 1::2] = 1

img_arr_conn = connectivity(img_arr_low, img_arr_high)

fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
ax1.imshow(img_arr_low, cmap='gray')
ax2.imshow(img_arr_high, cmap='gray')
ax3.imshow(img_arr_conn, cmap='gray')


In [ ]:
img = Image.open('img/lebrun_strad.jpg')

img_arr = np.asarray(img)
img_arr = rgb2gray(img_arr)

h, w = img_arr.shape
ratio = w / h
new_h = 300
new_w = new_h * ratio

img_arr = resize(img_arr, (new_h, new_w))
print(img_arr.shape)

# Line Example
# img_arr = np.zeros((10, 10))
# img_arr[5, 1:9] = 255

# Square Example
# img_arr = np.zeros((100, 100), dtype=np.float32)
# img_arr[30:70, 30:70] = 1.0

# Canny Edge Detection

In [ ]:
dx = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)

dy = np.array([
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
], dtype=np.float32)

img_gaussian = filters.gaussian(img_arr, 1.)
img_dx = ndi.correlate(img_gaussian, dx)
img_dy = ndi.correlate(img_gaussian, dy)
grad_norms = np.sqrt((img_dx**2 + img_dy**2))
angle_img = np.arctan(img_dy / img_dx)

supp_img = nonmaxima_suppression(grad_norms, angle_img)

In [ ]:
# Show the gradient norm image
fig = plt.figure(figsize=(6, 8))
ax1 = fig.add_subplot(111)
ax1.imshow(supp_img, cmap='gray')

In [ ]:
low_thresh = 0.1 * img_arr.max()
high_thresh = 0.2 * img_arr.max() #2.5 * low_thresh

# Low threshold -- considered "weak" edges
supp_img_low = supp_img.copy()
supp_img_low[supp_img < low_thresh] = 0
supp_img_low[supp_img >= low_thresh] = 1

# High threshold -- these are all guaranteed edge pixels
supp_img_high = supp_img.copy()
supp_img_high[supp_img < high_thresh] = 0
supp_img_high[supp_img >= high_thresh] = 1

# Eliminate nonzero pixels in high img from low img
supp_img_low = supp_img_low - supp_img_high

# Use connected components to combine strong and weak edges
canny_final = connectivity(supp_img_low, supp_img_high)

# Comparison with library version
canny_img = feature.canny(img_arr)

# fig = plt.figure(figsize=(8, 6))
# ax1 = fig.add_subplot(111)
# ax1.imshow(canny_final, cmap='gray')

# Comparison Figure
# fig = plt.figure(figsize=(8, 10))
# ax1 = fig.add_subplot(221)
# ax2 = fig.add_subplot(222)
# ax3 = fig.add_subplot(223)
# ax4 = fig.add_subplot(224)
# ax1.imshow(supp_img, cmap='gray')
# ax2.imshow(supp_img_low, cmap='gray')
# ax3.imshow(supp_img_high, cmap='gray')
# ax4.imshow(canny_final, cmap='gray')

# Dual Image Figure
fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.imshow(canny_img, cmap='gray')
ax2.imshow(canny_final, cmap='gray')